<a href="https://colab.research.google.com/github/nedlecky/CSC485B/blob/main/CSC485_135_PythagorasComplete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSC 485B Spring 2023: CSC485_135_PythagorasComplete using MLP
## Solving the Pythagoras problem using what we learned!
### Input the length of the two sides, ML computes hypotenuse, perimeter, and area
* SUNY Plattsburgh, Spring 2023
* Dr. Ned Lecky
* nleck001@plattsburgh.edu
* ned@lecky.com

Assignment:
* Follow the architectural idea from Class 04: Slides 17-18 as discussed in class
* Build your “Pythagoras dataset” to include 400 random triangles with sizes from 2cm to 2000cm on each side
* Split it into training and test portions using sklearn.model_selection.train_test_split
* Scale using std_scaler and figure out how to use a sklearn.pipeline so you don’t have to keep remembering to scale all of your Xs
* Get your super simple MLPs running as suggested on slide 17- Success is: max error < 0.5cm on hypotenuse, < 1cm on perimeter, and < 1cm^2 on area
* Now, make the data more “realistic” by adding small random errors to your dataset
  * Generate an exact dataset
  * Now make your training dataset by assuming there are small errors in side1 and side2 measurements
* Does your same code work when run on the exact data?



In [1]:
# Setup and Support Functions
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys

# This makes us reproducible (and we can adjust fixed_seed to get different results)
fixed_seed = 1

# Return n random floats between lo and hi as 1-column NumPy matrix
def rand_nlohi(n=1, lo=0, hi=1):
  # This is just a uniform distribution from lo to hi... we can adjust if appropriate in the future
  return (np.random.rand(n) * (hi - lo) + lo).reshape(-1,1)

# Often a good idea as long as we are keeping values near +/- 1... don't need exponential notation
np.set_printoptions(floatmode='fixed', precision=4, suppress=True)
# This will get us all 400 rows printed... which fails past 40 x 2 columns
np.set_printoptions(threshold=sys.maxsize)

# Simple numpy array print with optional push to file
def nprint(m, name='', also_write_file=False):
  print(f"{name} {m.shape} {m.dtype}")
  print(m)
  if also_write_file and name != '':
    fprint(m, name)

# Print numpy array to file (needs name)
def fprint(m, name=''):
  if name != '':
    with open(name, 'w') as f:
      print(f"{name} {m.shape} {m.dtype}", file=f)
      print(m, file=f)
  else:
    print('fprint needs a name!')

# Remove a file and don't complain if it doesn't exist
def remove_file(name):
  try:
    os.remove(name)
  except:
    return

remove_file('X')
remove_file('XX_scaled')
remove_file('Y')
remove_file('Y_pred')
remove_file('XY')
remove_file('X_train')
remove_file('X_train_scaled')
remove_file('X_test')
remove_file('Y_train')
remove_file('Y_test')
remove_file('Y_testY_pred')

remove_file('X1')
remove_file('X1_train')
remove_file('X1_train_scaled')
remove_file('X1_test')
remove_file('Y1')
remove_file('Y1_pred')
remove_file('Y1_train')
remove_file('Y1_test')
remove_file('X1Y1')
remove_file('Y1_testY1_pred')


remove_file('X2')
remove_file('X2_train')
remove_file('X2_train_scaled')
remove_file('X2_test')
remove_file('Y2')
remove_file('Y2_pred')
remove_file('Y2_train')
remove_file('Y2_test')
remove_file('X2Y2')
remove_file('Y2_testY2_pred')



# Make Test Data X
## 400 Triangles with side1 side2 spread from 2 to 2000 cm


In [2]:
# This is the full test input data for right triangles
# Reminder: Final input is the length of the two sides, output is length of hypotenuse, perimeter, and area
# x = [side1, side2]
# y = [hypotenuse, perimeter, area]
import math

np.random.seed(fixed_seed)

# Setup what you want to generate
N = 100
use_trivial_test_data = False  # Force use of just 2 easy triangles
make_lengths_random = True     # Random or just stepped in size
split_data = True             # Split test/train 50/50? (else test=train)

shortest_side = 2
longest_side = 2000
raw_scale = np.array([0.01, 0.01])

# Generate X
if(use_trivial_test_data):
  # Trivial Test Data
  X = np.array([
    [math.sqrt(2)/2, math.sqrt(2)/2],
    [1, 1]
  ])
else:
  if(make_lengths_random):
    # This makes random X
    side1 = rand_nlohi(N, shortest_side, longest_side)
    side2 = rand_nlohi(N, shortest_side, longest_side)
  else:
    # OR: This makes more regular X, all equilateral triangles
    side1 = np.arange(shortest_side, longest_side, longest_side/N).reshape(-1,1)
    side2 = np.arange(shortest_side, longest_side, longest_side/N).reshape(-1,1)

  X_raw = np.hstack([side1, side2])
  fprint(X_raw,'X_raw')
  X = X_raw * raw_scale

fprint(X,'X')


# Make expected Y

In [3]:
# Now let's compute the FULL X,Y expected results
# Reminder: We tell you the length of the two sides, you compute length of hypotenuse, perimeter, and area
# x = [side1, side2]
# y = [hypotenuse, perimeter, area]
from sklearn.model_selection import train_test_split

hypotenuse = np.sqrt(np.square(X[:,0:1]) + np.square(X[:,1:2]))
perimeter = X[:,0:1] + X[:,1:2] + hypotenuse
area = (X[:,0:1] * X[:,1:2]) / 2.
Y = np.hstack([hypotenuse, perimeter, area])
fprint(Y,'Y')
fprint(np.hstack([X,Y]), 'XY')

# Optional noise in Y
# Not tested yet!
# Just bump all up or down by up to 1%
#Y = Y * (100. + (np.random.rand(Y.shape[0],Y.shape[1])-0.5))/100.


# MLP Core Functions

In [4]:
# The MLP core functions

from sklearn import preprocessing
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

def scale(X):
  scaler = preprocessing.StandardScaler().fit(X)
  X_scaled = scaler.transform(X)
  return scaler, X_scaled

def trainMLP(X, Y, hidden_layer_sizes=(10), activation='relu', max_iter=10000):
  # Not going to tune hyperparameters here... but we could!  
  regr = MLPRegressor(solver='lbfgs', alpha=1e-5,
                     hidden_layer_sizes=hidden_layer_sizes,
                     activation=activation,
                     max_iter=max_iter,
                     random_state=1,
                     verbose=True)
  regr.fit(X, Y)
  return regr

def evaluate(Y, Y_pred):
  print(f"Mean squared error: {mean_squared_error(Y, Y_pred):.2f}")
  print(f"Mean absolute error: {mean_absolute_error(Y, Y_pred):.2f}")
  print(f"Mean absolute percentage error: {mean_absolute_percentage_error(Y, Y_pred):.2f}")

  # Add the Pandas describe()
  df = pd.DataFrame(data = Y_pred - Y)
  print(df.describe())

def test(regr, scaler, X, Y):
  # The reshape below forces Y_pred to come back as 1 column if there is ony 1 output
  Y_pred = regr.predict(scaler.transform(X)).reshape(Y.shape[0],-1)

  evaluate(Y, Y_pred)

  return Y_pred


# Let's try going from X to Y directly

In [5]:
# And here are the full train/test sets if we want to try the old way
if(split_data):
  (X_train, X_test, Y_train, Y_test) = train_test_split(X, Y, test_size=0.5, random_state=1)
else:
  # OR we can have train=test!
  X_train = X_test = X
  Y_train = Y_test = Y

fprint(X_train, 'X_train')
fprint(X_test, 'X_test')
fprint(Y_train, 'Y_train')
fprint(Y_test, 'Y_test')


In [6]:
# Make a scaled set of all of X
(scaler_X, X_scaled) = scale(X)

# Initial try to train on the whole problem
(scaler_X_train, X_train_scaled) = scale(X_train)
regr_all = trainMLP(X_train_scaled, Y_train)
Y_pred = test(regr_all, scaler_X_train, X_test, Y_test)

fprint(np.hstack([X, X_scaled]),'XX_scaled')
fprint(X_train_scaled,'X_train_scaled')
fprint(Y_pred,'Y_pred')
fprint(np.hstack([Y_test, Y_pred]),'Y_testY_pred')


Mean squared error: 6.89
Mean absolute error: 1.41
Mean absolute percentage error: 0.35
               0          1          2
count  50.000000  50.000000  50.000000
mean   -0.179803  -0.154289   2.326076
std     0.586197   0.584334   3.850286
min    -1.307375  -1.313545  -5.565820
25%    -0.619224  -0.573361  -0.086377
50%    -0.059719  -0.010872   1.273566
75%     0.224786   0.297260   5.250665
max     0.718926   0.662807  10.801959


# Make X1 and Y1 for First MLP

In [7]:
# We want X1 to be [side1 side2 side1^2 side2^2] BUT the side1,2 should be scaled first
# And Y1 to be [hypotenuse^2, area]

# Not we build X1 from X_scaled
X1 = np.hstack([X_scaled, np.square(X_scaled)])
Y1 = np.hstack([np.square(Y[:, 0:1]), Y[:,2:3]])

fprint(X1, 'X1')
fprint(Y1, 'Y1')
fprint(np.hstack([X1,Y1]), 'X1Y1')

if(split_data):
  (X1_train, X1_test, Y1_train, Y1_test) = train_test_split(X1, Y1, test_size=0.5, random_state=1)
else:
  # OR we can have train=test!
  X1_train = X1_test = X1
  Y1_train = Y1_test = Y1

fprint(X1_train, 'X1_train')
fprint(X1_test, 'X1_test')
fprint(Y1_train, 'Y1_train')
fprint(Y1_test, 'Y1_test')


In [8]:

(scaler1, X1_train_scaled) = scale(X1_train)

regr1 = trainMLP(X1_train_scaled, Y1_train)
Y1_pred = test(regr1, scaler1, X1_test, Y1_test)

fprint(X1_train_scaled,'X1_train_scaled')
fprint(Y1_pred,'Y1_pred')
fprint(np.hstack([Y1_test, Y1_pred]),'Y1_testY1_pred')


Mean squared error: 12.01
Mean absolute error: 1.98
Mean absolute percentage error: 1.03
               0          1
count  50.000000  50.000000
mean    0.011682   1.520563
std     0.147186   4.704073
min    -0.380381  -7.607224
25%    -0.039016  -1.717609
50%     0.029142   0.584716
75%     0.068791   4.971727
max     0.455576  13.723027


# Make X2 and Y2 for Second MLP

In [9]:
# We want X2 to be [side1 side2 side3] = [X1 , sqrt(Y1[:0])]
# And Y2 to be [perim]
X2 = np.hstack([X, np.sqrt(Y1[:,0:1]).reshape(-1,1)])
Y2 = Y[:,1:2]

fprint(X2, 'X2')
fprint(Y2, 'Y2')
fprint(np.hstack([X2,Y2]), 'X2Y2')

if(split_data):
  (X2_train, X2_test, Y2_train, Y2_test) = train_test_split(X2, Y2, test_size=0.5, random_state=1)
else:
  # OR we can have train=test!
  X2_train = X2_test = X2
  Y2_train = Y2_test = Y2

fprint(X2_train, 'X2_train')
fprint(X2_test, 'X2_test')
fprint(Y2_train, 'Y2_train')
fprint(Y2_test, 'Y2_test')


In [10]:
# Train and Test
(scaler2, X2_train_scaled) = scale(X2_train)
regr2 = trainMLP(X2_train_scaled, Y2_train)
Y2_pred = test(regr2, scaler2, X2_test, Y2_test).reshape(-1,1)

fprint(X2_train_scaled,'X2_train_scaled')
fprint(Y2_pred,'Y2_pred')
fprint(np.hstack([Y2_test, Y2_pred]),'Y2_testY2_pred')


Mean squared error: 0.00
Mean absolute error: 0.00
Mean absolute percentage error: 0.00
               0
count  50.000000
mean    0.000019
std     0.000018
min    -0.000025
25%     0.000011
50%     0.000017
75%     0.000031
max     0.000057


/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [11]:
# The complete 2-MLP solution is
# X --> [sqrt(Y1[:0]) Y2[:0] Y1[:1]]
Y_pred12 = np.hstack([np.sqrt(Y1_pred[:,0:1]), Y2_pred[:,0:1], Y1_pred[:,1:2]])

fprint(Y_pred12,'Y_pred12')
Y_predY_pred12 = np.hstack([Y_pred, Y_pred12])
fprint(Y_predY_pred12,'Y_predY_pred12')

evaluate(Y_pred, Y_test)
evaluate(Y_pred12, Y_test)



Mean squared error: 6.89
Mean absolute error: 1.41
Mean absolute percentage error: 0.07
               0          1          2
count  50.000000  50.000000  50.000000
mean    0.179803   0.154289  -2.326076
std     0.586197   0.584334   3.850286
min    -0.718926  -0.662807 -10.801959
25%    -0.224786  -0.297260  -5.250665
50%     0.059719   0.010872  -1.273566
75%     0.619224   0.573361   0.086377
max     1.307375   1.313545   5.565820
Mean squared error: 8.00
Mean absolute error: 1.29
Mean absolute percentage error: 0.23
               0          1          2
count  50.000000  50.000000  50.000000
mean    0.001601  -0.000019  -1.520563
std     0.013177   0.000018   4.704073
min    -0.013923  -0.000057 -13.723027
25%    -0.002263  -0.000031  -4.971727
50%    -0.000680  -0.000017  -0.584716
75%     0.001116  -0.000011   1.717609
max     0.085287   0.000025   7.607224


# Try with Keras / TensorFlow??
## This is just an initial exploration- we have a long way to go to getting this to work well!

In [12]:
import keras
#from tensorflow import keras
from keras import layers

inputs = keras.Input(shape=(2,))
# Initial Design
x1 = layers.Dense(12, activation="relu")(inputs)
x2 = layers.Dense(30, activation="relu")(x1)
x3 = layers.Dense(30, activation="relu")(x2)
outputs = layers.Dense(3, activation='linear')(x3)    

# Simpler
#x1 = layers.Dense(10, activation="relu")(inputs)
#x2 = layers.Dense(20, activation="relu")(x1)
#outputs = layers.Dense(3, activation='linear')(x2)    

model = keras.Model(inputs=inputs, outputs=outputs, name="qKerasMLPAligner_model")
model.summary()
model.compile(
  loss='mse',
  optimizer='adam',
  metrics=["mse", "mae"],
)

batch_size = min(X_train.shape[0], 50)
history = model.fit(X_train, Y_train, epochs=1000, batch_size=batch_size,  verbose=1)#2)

Y_predk = model.predict(X_test)

fprint(Y_predk,'Y_predk')
fprint(np.hstack([Y_test, Y_predk]),'Y_testY_predk')



Model: "qKerasMLPAligner_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 2)]               0         
                                                                 
 dense (Dense)               (None, 12)                36        
                                                                 
 dense_1 (Dense)             (None, 30)                390       
                                                                 
 dense_2 (Dense)             (None, 30)                930       
                                                                 
 dense_3 (Dense)             (None, 3)                 93        
                                                                 
Total params: 1,449
Trainable params: 1,449
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
1/1 [========================